In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [23]:
## Fetching the data
housing=fetch_california_housing()

In [10]:
## Creating a dataframe
housing_df = pd.DataFrame(housing.data, columns=housing.feature_names)
housing_df['Price'] = housing.target

In [11]:
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Train test split the data, model training, prediction and evaluation

In [24]:
## Important library used in mlflow
from urllib.parse import urlparse
import os

## Divide my data into independent and dependent variables
X=housing_df.drop('Price', axis=1)
y=housing_df['Price']

## Splitting the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
## Hyperparameter tuning using GridSearchCV
def hyperparameter_tuning(X_train,y_train,param_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2,scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    return grid_search

In [27]:
from mlflow.models.signature import infer_signature

signature = infer_signature(X_train, y_train)

## Defining the hyperparameters

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5,10,None],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [2, 5]
}

## Start the mlflow experiment
with mlflow.start_run():
    ## Hyperparameter tuning
    grid_search = hyperparameter_tuning(X_train,y_train,param_grid)
    ## get the best parameters
    best_model = grid_search.best_estimator_

    ## Evaluate the model 
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print('Mean Squared Error:', mse)

    ## Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_metric("mse", mse)


    ## tracking uri
    mlflow.set_tracking_uri(" http://127.0.0.1:5000")
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    

    ## If the tracking uri is file , we will save the model with the signature or else we will save the model with the registered model name
    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model, "model", registered_model_name="California_Housing_Price")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)

    print("Best Hyperparameters Found: ", grid_search.best_params_)
    print("Mse", mse)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.6s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  10.7s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   5.2s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  10.8s
[CV] END max_depth=5, min_samples_leaf=

2024/10/22 23:32:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'California_Housing_Price' already exists. Creating a new version of this model...
2024/10/22 23:32:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: California_Housing_Price, version 2
Created version '2' of model 'California_Housing_Price'.
2024/10/22 23:32:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run dashing-rook-890 at:  http://127.0.0.1:5000/#/experiments/0/runs/875c91a4458f4b41ba04c992a2fbf1ba.
2024/10/22 23:32:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at:  http://127.0.0.1:5000/#/experiments/0.


Best Hyperparameters Found:  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
Mse 0.2504093047820202
